In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.models import resnet50
from torchvision.datasets import ImageFolder

In [3]:
class DinosaurDataset(Dataset):
    def __init__(self, folder_directory, transform=None):
        self.data = ImageFolder(folder_directory, transform=transform)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

In [4]:
transform = transforms.Compose([
    transforms.Resize((250, 250)),
    transforms.ToTensor(),
])

dino_dataset = DinosaurDataset('dinosaurpics_scraped',transform)

In [5]:
# splits - 70% training, 15% testing, 15% testing
train_size = int(0.7 * len(dino_dataset))  
val_size = int(0.15 * len(dino_dataset))   
test_size = len(dino_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dino_dataset, [train_size, val_size, test_size]
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
# ResNet50 model for image
model = resnet50()

In [7]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

C:\Users\ethan\anaconda3\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [9]:
model.eval()  
def calculate_accuracy(model, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

# Calculate accuracy on validation set
val_accuracy = calculate_accuracy(model, val_loader)
print(f'Validation Accuracy: {val_accuracy:.4f}')

Validation Accuracy: 0.3333
